# Review Preprocessing

### 패키지 import

In [1]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer  # 어근추출
from nltk.tokenize import RegexpTokenizer  # 정규표현식을 사용하여 단어 토큰화를 제공
from nltk.corpus import stopwords  # 불용어 정의
import os
import glob
import re

### Review 파일 경로 설정 

In [5]:
default_path = r'C:\Users\SOJINSOO\여행고수\food_review(번역)' # 기본 경로 설정

os.chdir(default_path) # 기본 경로로 경로 변경

### 모든 Review(csv 파일)를 DataFrame 형태로 통합

In [6]:
countries = os.listdir() # 현재 경로에 존재하는 폴더명(국가명)

country_list = []
food_review_total = pd.DataFrame() # 통합할 변수 선언 및 초기화

# 모든 리뷰(csv파일)를 DataFrame 형태로 통합('국가', 'label(식당명, 호텔명, 관광지명)' 컬럼 추가)
for country in countries:
    os.chdir(default_path)
    os.chdir(country)
    os.chdir('review')
    os.chdir('final_review')
    food_path = glob.glob('*.csv')
    #확장자명을 비교해 csv 리스트들을 만든다.
    for path in food_path:
        try:
            food_review = pd.read_csv(path, encoding = "UTF-8")
            # 식당명 컬럼 추가
            food_review['label'] = os.path.splitext(os.path.basename(path))[0]
            # 국가 컬럼 추가
            food_review['country'] = country
            country_list.append(country)
            # DataFrame 형태로 통합
            food_review_total = pd.concat([food_review_total, food_review])
        except: pass
        
# 모든 리뷰를 통합한 DataFrame Index 초기화
food_review_total = food_review_total.reset_index(drop = True) 
food_review_total

,title,rating,content,date,lang,label,country
0,아미고 루프트 탑 바,5.0,"Located in the heart of Phnom Penh, the Pong N...",2018년 6월 17일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
1,좋은 음식,5.0,It's a great place for dinner and fun with fam...,2019년 6월 28일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
2,"좋은 접대, 음식 및 음료",5.0,"It is best for all family, friends and couples...",2019년 6월 28일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
3,권하다,5.0,The roads and the environment are pretty. The ...,2019년 6월 28일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
4,좋은 전망,5.0,Choose this place if you want to meet the wond...,2019년 6월 28일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
5,스카이 바 정보,5.0,Good food and service. The atmosphere here is ...,2019년 5월 18일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
6,환상적인 도시 전망,5.0,This was not the restaurant we were looking fo...,2019년 5월 18일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
7,파티,4.0,Nice restaurant with nice view ??? It was my f...,2018년 12월 23일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
8,친구와 작별 인사,5.0,"Happily with friends, good food and service ??...",2018년 12월 23일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia
9,친구 동창회와 이별.,4.0,Good atmosphere and friendly service ??? I hop...,2018년 12월 23일,ko,restaurant_review_Amigo Restaurant & Sky Bar,cambodia


### 리뷰 텍스트 Preprocessing

In [22]:
# 이모지 정규표현식 패턴
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


food_review_total['content'] = food_review_total['content'].apply(lambda x: emoji_pattern.sub('', str(x))) # 이모지 제거
food_review_total['content'] = food_review_total['content'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x))) # 영어를 제외한 문자 제거
food_review_total['content'] = food_review_total['content'].apply(lambda x: str(x).lower().split()) # 소문자로 변환
stops = set(stopwords.words('english')) # 불용어 사전
food_review_total['content'] = food_review_total['content'].apply(lambda x: [word for word in x if not word in stops]) # 불용어 처리
food_review_total['content'] = food_review_total['content'].apply(lambda x: ' '.join(x)) # 모든 리뷰 공백으로 통ㄹ합

# 필요한 컬럼만 추출
column_list = ['label', 'content', 'country']
food_review_total = food_review_total[column_list]

# 고유 식당명 추출
food_review_totalstr.replace('restaurant_review_', '')

# 음식점별 리뷰를 통합하여 DataFrame 형태로 변환 
food_review_join = food_review_total.groupby(['label'])['content'].apply(lambda x:' '.join(list(x)))
df_food_review = pd.DataFrame(food_review_join)
df_food_review = df_food_review.reset_index() # index 초기화

# 국가명 재생성
df_food_review['country'] = ""

for i in range(len(df_food_review)):
    df_food_review['country'][i] = list(food_review_total[food_review_total['label'] == df_food_review['label'][i]]['country'])[0]

df_food_review

,label,content,country
0,055 Lobster Seafood Restaurant,use option near sea close food fantastic unfor...,taiwan
1,1 ping (Flagship Store),nan nan nan nan nan nan nan nan nan nan nan na...,taiwan
2,1010 Hunan Cuisine - Fubei Store,second month row selected hunan dinner fuxing ...,taiwan
3,1010 Hunan Cuisine - Kaohsiung E-DA Store,amount la carte xiang restaurant relatively sm...,taiwan
4,1010 Hunan Cuisine - Nangang CTBC Financial Pa...,taste spicy good service good atmosphere come ...,taiwan
5,1010 Hunan Cuisine - Sinyi Eslite Store,due advance reservations take place quickly se...,taiwan
6,1010 Hunan Cuisine - Taichung Parkland Store,general enter restaurant chain desperate expec...,taiwan
7,1010 Hunan Cuisine - Tainan Ximen Mitsukoshi S...,food place miss preserved pepper eggs also ord...,taiwan
8,1010 Hunan Cuisine - Tianmu Mitsukoshi Store,first time ate restaurant environment comforta...,taiwan
9,1010 Hunan Cuisine - Xihu Store,xiangxi lake restaurant restaurant like much t...,taiwan


### 음식점별 리뷰 텍스트 통합 DataFrame Pickle Dump

In [7]:
import pickle

with open(r"C:\Users\SOJINSOO\여행고수\df_food_review.txt", "wb") as f:
    pickle.dump(df_food_review, f)

## City(도시) 컬럼을 추가하기 위한 Info 정보와 JOIN

### info 파일 기본 경로 설정

In [ ]:
default_path = r'C:\Users\SOJINSOO\여행고수\여행고수음식점 백업(list,info)' # 기본 경로

os.chdir(default_path)

### 모든 국가, 도시의 info 정보를 통합한 DataFrame 생성('국가', '도시' 컬럼 추가)

In [ ]:
food_info_total = pd.DataFrame() # 통합할 변수 선언 및 초기화

tokenizer = RegexpTokenizer('[\w]+')  # \w : 단어 영문자+숫자+_(밑줄) [0-9a-zA-Z_]
stop_words = stopwords.words('english')  # 불용어 사전

food_info_total = pd.DataFrame() # 통합할 변수 선언 및 초기화
countries = os.listdir()

# 모든 국가, 도시의 Info 정보를 DataFrame으로 통합
for country in countries:
    os.chdir(default_path)
    os.chdir(country)
    cities = os.listdir()
    
    for city in cities:
        os.chdir(default_path + '\\' + country + "\\" + city)
        os.chdir('info')
        food_path = glob.glob('restaurant_scraped_list.csv') #확장자명을 비교해 csv 리스트들을 만든다.
        
        for path in food_path:
            food_info = pd.read_csv(path, encoding = "UTF-8")
            food_info = pd.DataFrame(food_info)
            # 국가 컬럼 추가
            food_info['country'] = country
            # 도시 컬럼 
            food_info['city'] = city
            country_list.append(country)
            # DataFrame 형태로 통합
            food_info_total = pd.concat([food_info_total, food_info])
            
# Info 정보 통합 DataFrame Index 초기화
food_info_total = food_info_total.reset_index(drop = True)

# 필요한 컬럼만 추출
column_list = ['001_상호명', 'country', 'city', '002_전체평점']
food_info_total = food_info_total[column_list]

### Review DataFrame과 Info DataFrame Join

In [28]:
# Review DataFrame과 Info DataFrame Join
food_review_merge = pd.merge(df_food_review, food_info_total, how='inner', left_on=['country', 'label'], right_on = ['country', '001_상호명'])

# 필요한 컬럼만 추출
column_list = ['label', 'city', 'country', 'content', '002_전체평점']
food_review_merge = food_review_merge[column_list]
# 컬럼명 재정의
food_review_merge.rename({'label': 'restaurant', '002_전체평점': "rating"}, axis = 'columns', inplace = True)
food_review_merge

,restaurant,city,country,content,rating
0,055 Lobster Seafood Restaurant,hualien,taiwan,use option near sea close food fantastic unfor...,3.5
1,1 ping (Flagship Store),beitou,taiwan,nan nan nan nan nan nan nan nan nan nan nan na...,4.0
2,1 ping (Flagship Store),beitou,taiwan,nan nan nan nan nan nan nan nan nan nan nan na...,4.0
3,1 ping (Flagship Store),newtaipei,taiwan,nan nan nan nan nan nan nan nan nan nan nan na...,4.0
4,1 ping (Flagship Store),sanxia,taiwan,nan nan nan nan nan nan nan nan nan nan nan na...,4.0
5,1 ping (Flagship Store),taipei,taiwan,nan nan nan nan nan nan nan nan nan nan nan na...,4.0
6,1 ping (Flagship Store),wanli,taiwan,nan nan nan nan nan nan nan nan nan nan nan na...,4.0
7,1010 Hunan Cuisine - Fubei Store,beitou,taiwan,second month row selected hunan dinner fuxing ...,4.0
8,1010 Hunan Cuisine - Fubei Store,newtaipei,taiwan,second month row selected hunan dinner fuxing ...,4.0
9,1010 Hunan Cuisine - Fubei Store,sanxia,taiwan,second month row selected hunan dinner fuxing ...,4.0


### City(도시) 구분까지 완료된 최종 DataFrame Pickle Dump

In [29]:
import pickle

with open("C:\\Users\SOJINSOO\\여행고수\\food_review_merge.txt", "wb") as f:
    pickle.dump(food_review_merge, f)